In [1]:
import os
from dotenv import load_dotenv

In [2]:
! pip install tiktoken

In [3]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser

In [4]:
load_dotenv()

True

In [5]:
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0
)

embed_model = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_EMBEDDING_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_EMBEDDING_API_KEY"),
    openai_api_version=os.getenv("AZURE_OPENAI_EMBEDDING_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
)

In [6]:
embed_model

AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7ea506fd5ff0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7ea506fd65c0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-3-small', openai_api_version='2023-05-15', openai_api_base=None, openai_api_type='azure', openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=2048, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True, azure_endpoint='https://azureaitestingteam.openai.azure.com/', azure_ad_token=None, azure_ad_token_provider=None, azure_ad_async_token_provider=None, validat

In [7]:
! pip install pinecone-client

In [8]:
! pip install pinecone

In [9]:
from pinecone import Pinecone, ServerlessSpec

In [10]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [11]:
index_name = os.getenv("PINECONE_INDEX_NAME")

In [12]:
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )


In [13]:
index = pc.Index(index_name)

/home/sudip/PycharmProjects/hackathon-inf/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
! pip install langchain-pinecone

In [15]:
from langchain_pinecone import PineconeVectorStore

In [16]:
vector_store = PineconeVectorStore(index=index, embedding=embed_model)

In [17]:
# Feature Extractor
feature_extract_system_prompt = """
You are a structured feature extractor for a PL/SQL auditing application.

Your job is to analyze the current user message **along with the conversation history** to extract relevant features needed to fetch contextual information for auditing a PL/SQL script.

You must:

1. **Use Conversation Context**:
   - Consider any previous messages where table names, column names, or client-specific identifiers were mentioned.
   - Resolve partial references in the current input using context from earlier messages.

2. **Identify Table Names**:
   - Look for any SQL statements (`CREATE TABLE`, `INSERT INTO`, `SELECT FROM`, `UPDATE`, `MERGE`, etc.) or explicit mentions of table names across the entire conversation.
   - Return **unique** table names in a list.

3. **Extract Column Names**:
   - From the user's PL/SQL script, conversation references, or partial descriptions, collect all unique column names of each tables.
   - Avoid including SQL keywords or expressions.

4. **Detect Client Identifier (if present)**:
   - Identify a `client_name` if a naming convention or message indicates a source client (e.g., user prompts that mention "client xyz").
   - If there's no indication in the message then infer the client abbreviation from the table name (e.g., table names like `eligibility_payer_1`, `payer_abc`) and use this abbreviation to infer client_name from list of payers specified below:
       [
            {{'name': 'United Healthcare', 'abbr': 'UHC'}},
            {{'name': 'Elevance Health', 'abbr': 'ANTM'}},
            {{'name': 'Aetna', 'abbr': 'AET'}},
            {{'name': 'Cigna Healthcare', 'abbr': 'CI'}},
            {{'name': 'Humana', 'abbr': 'HUM'}},
            {{'name': 'Kaiser Permanente', 'abbr': 'KP'}},
            {{'name': 'Centene Corporation', 'abbr': 'CNC'}},
            {{'name': 'Molina Healthcare', 'abbr': 'MOH'}},
            {{'name': 'Blue Cross Blue Shield Association', 'abbr': 'BCBSA'}},
            {{'name': 'Health Care Service Corporation', 'abbr': 'HCSC'}},
            {{'name': 'Highmark Health', 'abbr': 'HMI'}},
            {{'name': 'Geisinger Health Plan', 'abbr': 'GHP'}},
            {{'name': 'UPMC Health Plan', 'abbr': 'UPMC'}},
            {{'name': 'Independence Blue Cross', 'abbr': 'IBC'}},
            {{'name': 'Harvard Pilgrim Health Care', 'abbr': 'HPHC'}},
            {{'name': 'Tufts Health Plan', 'abbr': 'THP'}},
            {{'name': 'WellCare Health Plans', 'abbr': 'WCG'}},
            {{'name': 'Tricare', 'abbr': 'TRI'}},
            {{'name': 'Medicare', 'abbr': 'MCR'}},
            {{'name': 'Medicaid', 'abbr': 'MCD'}}
        ]

5. **Output Format**:
   Respond strictly in this JSON format with following key and value pairs:
    {{
        "tables": [
            {{
                 "table_name": "table_name_1",
                 "columns": ["column_1", "column_2", ...],
                 "client_name": "client_identifier" // or null if not found
            }},
            {{
                 "table_name": "table_name_2",
                 "columns": ["column_1", "column_2", ...],
                 "client_name": "client_identifier" // or null if not found
            }},
            {{
                 "table_name": "table_name_3",
                 "columns": ["column_1", "column_2", ...],
                 "client_name": "client_identifier" // or null if not found
            }}
        ],
        "markdown": "The plain text response formatted as Markdown"
    }}

6. If the table name starts with "standard_", set the client name of that table to "Default".

7. If you couldn't infer then just output with empty json object.

"""

# Context Prompt
context_prompt = """
{knowledgebase}
```
"""

In [18]:
template = {
    "feature_extractor": ChatPromptTemplate.from_messages(
        [("system", feature_extract_system_prompt), MessagesPlaceholder(variable_name="history"), ("human", "{question}")]
    ),
    "context": PromptTemplate.from_template(
        context_prompt
    )
}

In [19]:
# 2. Feature Extractor
feature_extractor_chain = template["feature_extractor"] | llm | JsonOutputParser()

In [20]:
# Users input
question = """
Please verify this script.

BEGIN
INSERT INTO standard_eligibility (
    member_id, first_name, last_name, date_of_birth, gender, address, city, state, zip_code, plan_id, group_number, coverage_start_date, coverage_end_date, coverage_level_code, subscriber_indicator, product_category_code, plan_type, market_category_code
)
SELECT
    mem_member_id AS member_id,
    first_name_id AS first_name,
    mem_last_name AS last_name,
    date_of_birth AS date_of_birth,
    phm_gender_val AS gender,
    clm_address AS address,
    city AS city,
    rx_state_val AS state,
    phm_zip_code_val AS zip_code,
    mem_plan_id AS plan_id,
    clm_group_number AS group_number,
    clm_coverage_start_date AS coverage_start_date,
    phm_coverage_end_date AS coverage_end_date,
    mem_coverage_level_code AS coverage_level_code,
    clm_subscriber_indicator AS subscriber_indicator,
    rx_product_category_code_val AS product_category_code,
    rx_plan_type AS plan_type,
    rx_market_category_code_val AS market_category_code
FROM eligibility_mcd;
END;
/
"""

In [21]:
print(template["feature_extractor"].invoke({"history": [], "question": question}).to_string())

System: 
You are a structured feature extractor for a PL/SQL auditing application.

Your job is to analyze the current user message **along with the conversation history** to extract relevant features needed to fetch contextual information for auditing a PL/SQL script.

You must:

1. **Use Conversation Context**:
   - Consider any previous messages where table names, column names, or client-specific identifiers were mentioned.
   - Resolve partial references in the current input using context from earlier messages.

2. **Identify Table Names**:
   - Look for any SQL statements (`CREATE TABLE`, `INSERT INTO`, `SELECT FROM`, `UPDATE`, `MERGE`, etc.) or explicit mentions of table names across the entire conversation.
   - Return **unique** table names in a list.

3. **Extract Column Names**:
   - From the user's PL/SQL script, conversation references, or partial descriptions, collect all unique column names of each tables.
   - Avoid including SQL keywords or expressions.

4. **Detect Cli

In [22]:
ai_message_feature_extractor = feature_extractor_chain.invoke({"history": [], "question": question})
ai_message_feature_extractor

{'tables': [{'table_name': 'standard_eligibility',
   'columns': ['member_id',
    'first_name',
    'last_name',
    'date_of_birth',
    'gender',
    'address',
    'city',
    'state',
    'zip_code',
    'plan_id',
    'group_number',
    'coverage_start_date',
    'coverage_end_date',
    'coverage_level_code',
    'subscriber_indicator',
    'product_category_code',
    'plan_type',
    'market_category_code'],
   'client_name': 'Default'},
  {'table_name': 'eligibility_mcd',
   'columns': ['mem_member_id',
    'first_name_id',
    'mem_last_name',
    'date_of_birth',
    'phm_gender_val',
    'clm_address',
    'city',
    'rx_state_val',
    'phm_zip_code_val',
    'mem_plan_id',
    'clm_group_number',
    'clm_coverage_start_date',
    'phm_coverage_end_date',
    'mem_coverage_level_code',
    'clm_subscriber_indicator',
    'rx_product_category_code_val',
    'rx_plan_type',
    'rx_market_category_code_val'],
   'client_name': 'Medicaid'}],
 'markdown': 'The script invol

In [23]:
features = ai_message_feature_extractor["tables"]
features

[{'table_name': 'standard_eligibility',
  'columns': ['member_id',
   'first_name',
   'last_name',
   'date_of_birth',
   'gender',
   'address',
   'city',
   'state',
   'zip_code',
   'plan_id',
   'group_number',
   'coverage_start_date',
   'coverage_end_date',
   'coverage_level_code',
   'subscriber_indicator',
   'product_category_code',
   'plan_type',
   'market_category_code'],
  'client_name': 'Default'},
 {'table_name': 'eligibility_mcd',
  'columns': ['mem_member_id',
   'first_name_id',
   'mem_last_name',
   'date_of_birth',
   'phm_gender_val',
   'clm_address',
   'city',
   'rx_state_val',
   'phm_zip_code_val',
   'mem_plan_id',
   'clm_group_number',
   'clm_coverage_start_date',
   'phm_coverage_end_date',
   'mem_coverage_level_code',
   'clm_subscriber_indicator',
   'rx_product_category_code_val',
   'rx_plan_type',
   'rx_market_category_code_val'],
  'client_name': 'Medicaid'}]

In [24]:
# Querying
def fetch_from_vector_store(features: dict) -> dict:
    results = vector_store.similarity_search(
        features["table_name"] + ", " + ", ".join(features["columns"]), # list of a table name and columns - comma separated
        k=len(features["columns"]) + 1, # this should be len of the list - feature extracted
        filter={"table_name": features["table_name"], "client_name": features["client_name"]}
    )
    
    data = {"ddl": "", "dd": [], "table_name": ""}
    for doc in results:
        if doc.metadata["file_name"].endswith(".sql"):
            data["ddl"] = doc.page_content.strip()
        else:
            data["dd"].append(doc.page_content)
        data["table_name"] = doc.metadata["table_name"]
    data["dd"] = "\n\n".join(data["dd"])
    return data


In [25]:
def format_context(features: dict) -> str:
    """Formats the fetched data into a context string."""
    # Prepare data from vector store
    list_data = []
    for feature in features:
        list_data.append(fetch_from_vector_store(feature))

    context = ""
    for item in list_data:
        context += f'Table {item["table_name"]} DDL: ```\n{item["ddl"]}\n```\n\nTable {item["table_name"]} Data Dictionary: ```\n{item["dd"]}\n```\n\n'

    return context.strip()


In [26]:
context = format_context(features)

In [27]:
print(context)

Table standard_eligibility DDL: ```
CREATE TABLE standard_eligibility (
    member_id VARCHAR2(50),
    first_name VARCHAR2(35),
    last_name VARCHAR2(60),
    date_of_birth DATE,
    gender CHAR(1),
    address VARCHAR2(100),
    city VARCHAR2(30),
    state CHAR(2),
    zip_code VARCHAR2(9),
    plan_id VARCHAR2(30),
    group_number VARCHAR2(50),
    coverage_start_date DATE,
    coverage_end_date DATE,
    coverage_level_code CHAR(3),
    subscriber_indicator CHAR(1),
    product_category_code CHAR(2),
    plan_type CHAR(3),
    market_category_code VARCHAR2(4)
);
```

Table standard_eligibility Data Dictionary: ```
column_name: plan_id
data_type: varchar
column_size: 30
column_description: Identifier for the member's plan.

column_name: plan_type
data_type: char
column_size: 3
column_description: Type of plan the member is enrolled in.

column_name: group_number
data_type: varchar
column_size: 50
column_description: Group number associated with the member's plan.

column_name: co